## Requirements

코드 실행에 필요한 라이브러리를 사전에 설치합니다.

In [ ]:
!pip install transformers
!pip install peft
!pip install datasets
!pip install fire
!pip install accelerate
!pip install bitsandbytes
!pip install sentencepiece

## 라이브러리 불러오기

코드 실행에 필요한 라이브러리를 불러옵니다.

In [ ]:
import os
import sys
import textwrap
from typing import List

import fire
import torch
import transformers
from datasets import load_dataset
from peft import (LoraConfig, PeftType, PromptTuningConfig, PromptTuningInit,
                  TaskType, get_peft_config, get_peft_model,
                  get_peft_model_state_dict, prepare_model_for_int8_training)
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from transformers import (AutoModelForCausalLM, AutoTokenizer,
                          LlamaForCausalLM, LlamaTokenizer,
                          default_data_collator,
                          get_linear_schedule_with_warmup)


## 하이퍼파라미터 정의

모델 훈련에 사용할 하이퍼파라미터를 정의합니다.

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
max_length = 64
learning_rate = 3e-2
num_epochs = 5  # 50
batch_size = 8

## 데이터 불러오기

구글 드라이브를 마운트하여 모델 학습, 검증 및 추론에 사용할 데이터를 불러옵니다.

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

data_dir = "/content/drive/MyDrive/Colab Notebooks/Data"
raw_train_data = pd.read_csv(os.path.join(data_dir, "train_data.csv"))
raw_val_data = pd.read_csv(os.path.join(data_dir, "val_data.csv"))
raw_test_data = pd.read_csv(os.path.join(data_dir, "test_data.csv"))

In [ ]:
raw_train_data.head(5)

,question,context,answer,other_answers
0,벨기에 언제 독립했어?,그러나 네덜란드를 견제하려는 영국과의 계속되는 전쟁에서 밀려나 세계의 상권을 영국에...,1831년,NaN
1,빈 회의가 열린게 언제야?,그러나 네덜란드를 견제하려는 영국과의 계속되는 전쟁에서 밀려나 세계의 상권을 영국에...,1815년,NaN
2,한라산에 있는 오름이 무슨 뜻이야?,점사질 토양을 제외하면 대부분 화산회토로 덮여있는게 특징이다. 여러 섬들이 위치해 ...,기생 화산,NaN
3,1999년 우리나라 대통령 이름이 뭐야?,2014년 2월 5일에는 서울대총장 추천위원회 추천위원에 선임됐다. 2015년 10...,김대중,NaN
4,후쿠자와 유키치는 뭐하는 사람이야?,인면수심의 홍종우를 논할 필요도 없지만은 김옥균의 시체가 경성으로 도착했을 때 종로...,일본 자유주의자,NaN


In [ ]:
train_data, val_data, test_data = [], [], []
for i in range(len(raw_train_data)):
    data = raw_train_data.iloc[i]
    tmp = {
        "instruction": data["question"],
        "input": data["context"],
        "output": data["answer"]
    }
    train_data.append(tmp)

for i in range(len(raw_val_data)):
    data = raw_val_data.iloc[i]
    tmp = {
        "instruction": data["question"],
        "input": data["context"],
        "output": data["answer"]
    }
    val_data.append(tmp)

for i in range(len(raw_test_data)):
    data = raw_test_data.iloc[i]
    tmp = {
        "instruction": data["question"],
        "input": data["context"],
        "output": data["answer"]
    }
    test_data.append(tmp)

In [ ]:
from pprint import pprint

print("==========train_data==========")
pprint(train_data[0])
print("\n==========val_data==========")
pprint(val_data[0])
print("\n==========test_data==========")
pprint(test_data[0])

==========train_data==========
{'input': '그러나 네덜란드를 견제하려는 영국과의 계속되는 전쟁에서 밀려나 세계의 상권을 영국에 빼앗기게 되었으며, 1793년 2월 '
          '1일, 프랑스는 네덜란드에 선전 포고하였다. 1810년에는 프랑스 혁명의 소용돌이에 휘말려 프랑스 영토가 되기도 하였다. '
          '그러나 1815년 빈 회의의 결과 네덜란드 왕국이 새로이 탄생하였다. 하지만 로마 가톨릭 국가인 벨기에와 개신교 국가인 '
          '네덜란드가 합병된 왕국이었기 때문에, 1831년 벨기에가 네덜란드로부터 독립하여 벨기에 왕국을 건국했다. 제2차 세계 '
          '대전에서는 중립을 지키지 못하고 나치 독일의 지배하에 들어갔다. 《안네의 일기》로 유명한 안네 프랑크도 나치 독일의 '
          '네덜란드 점령기의 인물 중 하나이다. 전후 네덜란드는 안정된 정치와 높은 경제 발전을 이루었다.',
 'instruction': '벨기에 언제 독립했어?',
 'output': '1831년'}

==========val_data==========
{'input': '1995년 5월 23일에 대한민국 서울에서 열린 아시아 올림픽 평의회(OCA) 총회에서 대한민국 부산이 중화 타이베이 '
          '가오슝을 누르고 2002년 아시안 게임 개최 도시로 선정되었다.',
 'instruction': '제14회 아시안 게임이 개최된 지역은?',
 'output': '부산'}

==========test_data==========
{'input': '제87조 (1) 법관은 자신의 법관 직무를 수행함에 있어서 독립적이다. (2) 법관은, 법률의 규정에 따라 평의회나 '
          '위원회가 처리할 수 없는 사법행정사무를 제외하고, 법률과 사무분담에 따라 자신의 권한에 속하는 모든 재판사무를 처리한다. '
          '(3) 판사의 담당업무는 연방법률에서 정한 충분한 시간 여유를 

## 모델 정의

LLaMA 모델을 정의합니다.

In [ ]:
BASE_MODEL = "decapoda-research/llama-7b-hf"

model = LlamaForCausalLM.from_pretrained(
    BASE_MODEL,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

tokenizer = LlamaTokenizer.from_pretrained(BASE_MODEL)

tokenizer.pad_token_id = (
    0  # unk. we want this to be different from the eos token
)
tokenizer.padding_side = "left"

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=True`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
def generate_prompt(data_point):
    return f"""아래는 작업을 설명하는 명령어입니다. 문맥에 맞게 요청을 적절히 완료하는 응답을 작성하세요.\n\n### 명령어: {data_point["instruction"]} ### 문맥: {data_point["input"]} ### 응답: {data_point["output"]}"""


def tokenize(prompt, add_eos_token=True):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=1024,
        padding=False,
        return_tensors=None,
    )
    if (
        result["input_ids"][-1] != tokenizer.eos_token_id
        and len(result["input_ids"]) < 1024
        and add_eos_token
    ):
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)

    result["labels"] = result["input_ids"].copy()

    return result

def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenize(full_prompt)
    return tokenized_full_prompt


In [ ]:
generate_prompt(train_data[0])

'아래는 작업을 설명하는 명령어입니다. 문맥에 맞게 요청을 적절히 완료하는 응답을 작성하세요.\n\n### 명령어: 벨기에 언제 독립했어? ### 문맥: 그러나 네덜란드를 견제하려는 영국과의 계속되는 전쟁에서 밀려나 세계의 상권을 영국에 빼앗기게 되었으며, 1793년 2월 1일, 프랑스는 네덜란드에 선전 포고하였다. 1810년에는 프랑스 혁명의 소용돌이에 휘말려 프랑스 영토가 되기도 하였다. 그러나 1815년 빈 회의의 결과 네덜란드 왕국이 새로이 탄생하였다. 하지만 로마 가톨릭 국가인 벨기에와 개신교 국가인 네덜란드가 합병된 왕국이었기 때문에, 1831년 벨기에가 네덜란드로부터 독립하여 벨기에 왕국을 건국했다. 제2차 세계 대전에서는 중립을 지키지 못하고 나치 독일의 지배하에 들어갔다. 《안네의 일기》로 유명한 안네 프랑크도 나치 독일의 네덜란드 점령기의 인물 중 하나이다. 전후 네덜란드는 안정된 정치와 높은 경제 발전을 이루었다. ### 응답: 1831년'

In [ ]:
import json

with open("train_data.json", "w") as f:
    json.dump(train_data, f)


In [ ]:
train_data = load_dataset("json", data_files="train_data.json")
train_data

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
train_data = train_data.map(generate_and_tokenize_prompt)
val_data = val_data.map(generate_andtokenize_prompt)

AttributeError: ignored